In [1]:
import os 
import getopt
import sys

import random
import re

import numpy as np
import h5py
import pickle
import random
import copy
import pandas as pd
import math 

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Input, Masking, Lambda, concatenate, Bidirectional, Dense, Dropout, Flatten, Conv1D,BatchNormalization,  MaxPooling1D, Bidirectional, GRU, TimeDistributed
import tensorflow as tf
from tensorflow import keras



np.random.seed(1337) # for reproducibility
vocab = ["A", "G", "C", "T"]
indices = tf.range(len(vocab), dtype = tf.int64)
table_init = tf.lookup.KeyValueTensorInitializer(vocab,indices)
table = tf.lookup.StaticVocabularyTable(table_init, 1)
record_defaults = [
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),  
    tf.constant([''], dtype=tf.string),
    tf.constant([''], dtype=tf.string),
]

# Nadav dataset

def data_reader(file, batch_size=100, n_parse_threads=4):
    """Method for reading the data in an optimized way, can be used inside model.fit()
    
    Args:
        file (_type_): path to csv file
        batch_size (int, optional): _description_. Defaults to 100.
        n_parse_threads (int, optional): _description_. Defaults to 4.

    Returns:
        dataset.batch: batch dataset object 
    """
    dataset = tf.data.TextLineDataset(file).skip(1)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    return dataset.batch(batch_size).prefetch(1)

def preprocess(record):
    """Preprocessing method of a dataset object, one-hot-encodes the data

    Args:
        record (_type_): _description_

    Returns:
        X (2D np.array): one-hot-encoded input sequence
        Y (1D np.array): MPRA measurements for each cell state
    """
    # Get fields from the data
    fields = tf.io.decode_csv(record, record_defaults=record_defaults)
    
    # One-hot-encode data
    chars = tf.strings.bytes_split(fields[0])
    chars_indeces = table.lookup(chars)
    X = tf.one_hot(chars_indeces, depth = len(vocab))
    
    # Combine y for each cell type into one vector 
    Y = tf.stack(fields[2:])
    
    # Replace missing values with -1
    Y= tf.where(tf.equal(Y,  "nan"), ["-1"], Y)
    Y= tf.where(tf.equal(Y,  "NA"), ["-1"], Y)
    Y = tf.strings.to_number(Y, tf.float32)
    return X,Y

# Get first item of the dataset to get the shape of the input data
for element in data_reader("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/upper_LibA_wide_pivot_all_states.csv"):
    input_shape = element[0].shape
    output_shape = element[1].shape
    
print(output_shape)
print(input_shape)

2023-11-20 11:38:12.247113: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-20 11:38:12.278490: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 11:38:12.278525: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 11:38:12.278552: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 11:38:12.284772: I tensorflow/core/platform/cpu_feature_g

(92, 7)
(92, 262, 4)


In [2]:
# CROSS VALIDATION (10 fold)
from sklearn.model_selection import train_test_split, KFold

whole_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/upper_LibA_wide_pivot_all_states.csv"
out_data = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/"
# Split the data in two partitions
whole_data = pd.read_csv(whole_data)
k = 10
kf = KFold(n_splits = k, shuffle = True, random_state = 2008)

o=1
# For each fold we split again to get the third partition
for i in kf.split(whole_data):
    
    # Extract data from fold
    train = whole_data.iloc[i[0]]
    test =  whole_data.iloc[i[1]]
    
    # Split into train, validation and test
    valid, validation = train_test_split(train, test_size=0.111, random_state=42)
    
    # Randomize motifs
    #train = randomize_motifs_and_augment(train, num_augmentations=100)
    #train["rnd_seq"] = train['rnd_seq'].str.lower() 
    #test = randomize_motifs_and_augment(test, num_augmentations=100)
    #test["rnd_seq"] = test['rnd_seq'].str.lower() 
    #validation = randomize_motifs_and_augment(validation, num_augmentations=100)
    #validation["rnd_seq"] = validation['rnd_seq'].str.lower() 
    
    train.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibA_wide_pivot_state3_train.csv", index=False)
    test.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibA_wide_pivot_state3_test.csv", index=False)
    validation.astype(str).to_csv(out_data+"all_states_CV"+str(o)+"_LibA_wide_pivot_state3_validation.csv", index=False)
    o+=1
    

In [3]:
train

,seq,CRS,State_1M,State_2D,State_3E,State_4M,State_5M,State_6N,State_7M
0,AGGACCGGATCAACTAAAAAAATTTTCCACGCTTTCTGCTTAGCAC...,LibA.Seq2921,0.274892,0.392522,0.335505,0.232392,0.608189,0.328340,0.210542
1,AGGACCGGATCAACTAAAAAAGTGGGGCTTATAGTGTCCCTACCAG...,LibA.Seq4611,0.009031,0.074728,0.132928,0.068860,NaN,0.241786,NaN
3,AGGACCGGATCAACTAAAAATAACCGACCGCCAGGTGATCTTCCCT...,LibA.Seq2597,0.096726,-0.033663,0.147136,0.099091,NaN,0.246414,NaN
4,AGGACCGGATCAACTAAAAATACCAGCTCAACGCATCAGGCCCCGA...,LibA.Seq3943,0.012427,-0.101503,-0.019468,-0.044385,NaN,0.049059,NaN
5,AGGACCGGATCAACTAAAAATATGAGTGGTCCGGAACTTGATTATG...,LibA.Seq9018,0.229506,0.205531,0.263750,0.151651,0.252245,0.313759,-0.040573
...,...,...,...,...,...,...,...,...,...
8487,AGGACCGGATCAACTTTTTTCTCCTCCGATACTGTACGTTACTCTT...,LibA.Seq9733,NaN,NaN,0.405855,NaN,NaN,NaN,NaN
8488,AGGACCGGATCAACTTTTTTGATGCGTAAAGTCTAGAACGCTGGAA...,LibA.Seq1868,0.034790,0.026781,-0.056556,-0.184727,-0.106380,-0.081248,-0.235026
8489,AGGACCGGATCAACTTTTTTGCTAACATGCCAAATGCGGTAACAGA...,LibA.Seq7500,NaN,-0.133864,-0.005879,NaN,NaN,NaN,NaN
8490,AGGACCGGATCAACTTTTTTTATTTCAGTTTGGTATTCCGAACGAC...,LibA.Seq6690,0.486946,0.683003,0.624277,0.506531,0.381732,0.655849,0.464873


In [5]:
# Here we initialize the df where each fold test prediction will be appended to
# the list containing the correlations of each fold is also initialized
corr_list = []

# We define a custom normalization layer to then compile on the model
class CustomNormalization(Layer):
    """Custom normalization layer that normalizes the output of the neural network"""
    def __init__(self, **kwargs):
        super(CustomNormalization, self).__init__(**kwargs)
        
    def build(self, input_shape):
        # Add trainable variables for mean and standard deviation
        self.mean = self.add_weight("mean", shape=(1,), initializer="zeros", trainable=True)
        self.stddev = self.add_weight("stddev", shape=(1,), initializer="ones", trainable=True)
        super(CustomNormalization, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs):
        # Normalize the inputs using the learned mean and standard deviation
        return (inputs - self.mean) / (self.stddev + 1e-8)

# We define the method to compute the pearson correlation between prediction and ground truth in the multi_head case
def pearson_correlation_multi_head(predictions, ground_truth, mask_value=-1):
    """Computes Pearson Correlation between predictions and ground truth for each column
    Args:
        predictions (np.array): 2D array of prediction values (N, 7)
        ground_truth (np.array): 2D array of ground truth values (N, 7)
        mask_value (float): Value in ground truth to be ignored in correlation computation

    Returns:
        correlations (np.array): 1D array of Pearson correlations for each column
    """
    # Ensure predictions and ground_truth have the same shape
    assert predictions.shape == ground_truth.shape, "Input shapes do not match"

    n_columns = predictions.shape[1]
    correlations = np.zeros(n_columns)

    for col in range(n_columns):
        x = predictions[:, col]
        y = ground_truth[:, col]

        # Exclude values in ground truth equal to mask_value
        valid_indices = (y != mask_value)
        x = x[valid_indices]
        y = y[valid_indices]

        if len(x) == 0 or len(y) == 0:
            # If no valid values, set correlation to NaN
            correlations[col] = np.nan
        else:
            # Calculate mean of x and y
            mean_x = np.mean(x)
            mean_y = np.mean(y)

            # Calculate the numerator and denominators of the correlation coefficient
            numerator = np.sum((x - mean_x) * (y - mean_y))
            denominator_x = np.sqrt(np.sum((x - mean_x) ** 2))
            denominator_y = np.sqrt(np.sum((y - mean_y) ** 2))

            # Calculate the correlation coefficient
            correlation = numerator / (denominator_x * denominator_y)
            correlations[col] = correlation

    return correlations



import matplotlib.pyplot as plt
%matplotlib inline
# Define plotting function of loss
def create_plots(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    plt.clf()
    

In [7]:
# Define a custom loss function
class MaskedMSE(tf.keras.losses.Loss):
    """Computes the MSE loss and prevents missing values backpropagation (previously replaced by -1.0)

    Args:
        tf (_type_): _description_
    """
    def __init__(self, mask_value=-1, **kwargs):
        super(MaskedMSE, self).__init__(**kwargs)
        self.mask_value = mask_value

    def call(self, y_true, y_pred):
        # Create a mask for valid elements (not equal to the specified mask_value)
        mask = tf.math.not_equal(y_true, self.mask_value)

        # Compute MSE loss only for valid elements
        loss = tf.where(mask, tf.square(y_true - y_pred), 0.0)

        # Calculate the mean loss
        mean_loss = tf.reduce_sum(loss) / tf.reduce_sum(tf.cast(mask, dtype=tf.float32))

        return mean_loss

In [8]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

df_test_10folds  = pd.DataFrame(columns=['State_3E',
                                         "seq",
                                         "avg_prediction_State_1M",
                                         "avg_prediction_State_2D",
                                         "avg_prediction_State_3E",
                                         "avg_prediction_State_4M",
                                         "avg_prediction_State_5M",
                                         "avg_prediction_State_6N",
                                         "avg_prediction_State_7M"])
# We iterate through each of the train folds to train, test and validate the model

for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    correlation_ensemble = []
    # Initialize vector containing predictions
    
    for i in range(1,11):
        
        tf.random.set_seed(i)
        
        # Define and compile model
        inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
        layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
        layer = Dropout(0.5)(layer)
        layer = BatchNormalization()(layer)
        layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
        layer = Dropout(0.3)(layer)
        layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
        layer = BatchNormalization()(layer)
        layer = Dropout(0.5)(layer)
        layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
        layer = BatchNormalization()(layer)
        layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
        layer = Dropout(0.5)(layer)
        layer = Flatten()(layer)
        layer = Dense(300, activation='sigmoid')(layer)  # 300
        layer = Dropout(0.5)(layer)
        layer = Dense(200, activation='sigmoid')(layer)  # 300
        predictions = Dense(7, activation='linear')(layer)
        norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

        model = Model(inputs=inputs, outputs=norm_predictions)
        model.summary()

        # compile model
        model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                    loss=MaskedMSE(mask_value=-1),
                    metrics=["mse"]
                    )

        # Run model
        history=model.fit(data_reader(input_path_train, batch_size=100),
                                epochs=20,
                                validation_data=data_reader(input_path_valid,batch_size=500),
                                callbacks=None,
                                verbose=1)    

        # We predict the test data
        predicted = model.predict(data_reader(input_path_test, batch_size=500))

        # Sum the predicted values (will be averaged at the end of the 10 iterations)
        predictions_sum =  np.add(predictions_sum, predicted)
        
        
        # We reed the data in the same order to compute the correlation score
        test_data = data_reader(input_path_test,batch_size=500)
        test_tensor = np.empty(shape=[0,7])
        for batch in test_data:
            test_tensor = np.append(test_tensor, batch[1], axis=0)
            
        # Append correlation coefficient and append to previous
        corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
        correlation_ensemble.append(corr_coefficient)
        avg_corr_coefficient = 1
        
        
    # We fill the dataframe with predictions and fold annotation
    
    predicted = predictions_sum/10 # We divide by the number of iterations
    
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient

    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)    
    



# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout (Dropout)           (None, 256, 250)          0         
                                                                 
 batch_normalization (Batch  (None, 256, 250)          1000      
 Normalization)                                                  
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_1 (Bat  (None, 249, 250)          1000      
 chNormalization)                                            

2023-11-20 11:39:53.569482: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-20 11:39:53.572904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2023-11-20 11:39:53.695568: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-11-20 11:39:55.827588: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f57af5018d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-20 11:39:55.827624: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-11-20 11:39:55.831741: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-11-20 11:39:55.923239: I ./tensorflow/compiler/jit/device_comp

     77/Unknown - 8s 23ms/step - loss: 0.1051 - mse: 0.3989

2023-11-20 11:40:00.008173: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10650941850346279087
2023-11-20 11:40:00.008263: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 11998734545834586153


77/77 [==============================] - 9s 40ms/step - loss: 0.1051 - mse: 0.3989 - val_loss: 0.0526 - val_mse: 0.3646
Epoch 2/20
 5/77 [>.............................] - ETA: 1s - loss: 0.0499 - mse: 0.3719

...

2023-11-20 12:44:31.851121: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 14889894778614484549


Model: "model_98"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 262, 4)]          0         
                                                                 
 conv1 (Conv1D)              (None, 256, 250)          7250      
                                                                 
 dropout_490 (Dropout)       (None, 256, 250)          0         
                                                                 
 batch_normalization_392 (B  (None, 256, 250)          1000      
 atchNormalization)                                              
                                                                 
 conv2 (Conv1D)              (None, 249, 250)          500250    
                                                                 
 batch_normalization_393 (B  (None, 249, 250)          1000      
 atchNormalization)                                       

2023-11-20 12:44:36.928637: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 10650941850346279087
2023-11-20 12:44:36.928702: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 1835164627852203489
2023-11-20 12:44:36.928723: I tensorflow/core/framework/local_rendezvous.cc:421] Local rendezvous recv item cancelled. Key hash: 166672169564074572


77/77 [==============================] - 2s 22ms/step - loss: 0.0358 - mse: 0.3282 - val_loss: 0.0588 - val_mse: 0.3514
Epoch 3/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0296 - mse: 0.3214 - val_loss: 0.0568 - val_mse: 0.3345
Epoch 4/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0269 - mse: 0.3211 - val_loss: 0.0573 - val_mse: 0.3244
Epoch 5/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0259 - mse: 0.3193 - val_loss: 0.0582 - val_mse: 0.3175
Epoch 6/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0242 - mse: 0.3157 - val_loss: 0.0588 - val_mse: 0.3146
Epoch 7/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0232 - mse: 0.3140 - val_loss: 0.0558 - val_mse: 0.3044
Epoch 8/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0227 - mse: 0.3139 - val_loss: 0.0425 - val_mse: 0.3047
Epoch 9/20
77/77 [==============================] - 2s 22ms/step - loss: 0.0226 - mse: 0.3141 

,State_3E,seq,avg_prediction_State_1M,avg_prediction_State_2D,avg_prediction_State_3E,avg_prediction_State_4M,avg_prediction_State_5M,avg_prediction_State_6N,avg_prediction_State_7M,CRS,State_1M,State_2D,State_4M,State_5M,State_6N,State_7M,fold,partition
0,-0.005304,AGGACCGGATCAACTAAACAACTCAAACAAGGGCTAATATAACCCA...,0.090697,0.103051,0.088653,0.063847,0.049351,0.077512,0.048142,LibA.Seq7829,0.282666,0.123976,0.100559,NaN,-0.272353,NaN,10,test
1,0.236881,AGGACCGGATCAACTAAACACTAGTCATACTTAAAAATTGCAAGGA...,0.036217,0.045835,0.036513,0.022633,0.009836,0.011005,0.018444,LibA.Seq271,0.145643,0.086300,0.109373,NaN,0.088780,NaN,10,test
2,-0.086491,AGGACCGGATCAACTAAACAGGTTCTGACGTATGCTCCTCTATGGA...,0.027676,0.033017,0.017762,0.022750,0.009217,-0.010723,0.017712,LibA.Seq4548,-0.116828,0.011797,-0.046919,NaN,-0.181907,NaN,10,test
3,-0.087684,AGGACCGGATCAACTAAACCCGAGCCTGCCTAGCCCTAGCTTCTCT...,0.047688,0.067234,0.062880,0.041293,0.019733,0.049192,0.030736,LibA.Seq4582,0.015043,-0.052275,0.021085,0.069787,-0.053224,0.088238,10,test
4,0.371111,AGGACCGGATCAACTAAACGGAGCAGAGTTAGTGTCAGGTCAAAAA...,0.079280,0.085112,0.060707,0.060935,0.038236,0.069176,0.020441,LibA.Seq2863,0.344367,0.244331,0.145184,NaN,0.259818,NaN,10,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8487,-0.076574,AGGACCGGATCAACTTTTGGTCGGTTGACGGTCGCCTTGATTATTC...,0.053900,0.070003,0.045463,0.018510,-0.001826,0.030140,0.030768,LibA.Seq4154,-0.016039,-0.027890,-0.047924,-0.017993,-0.174528,-0.035544,10,test
8488,0.369648,AGGACCGGATCAACTTTTTCAGTGAAAGATCACCGCGGGATCTCAC...,0.308035,0.312025,0.240698,0.221484,0.236020,0.418047,0.216684,LibA.Seq8531,0.410579,0.461241,0.251085,0.337580,0.565330,0.273966,10,test
8489,-0.098525,AGGACCGGATCAACTTTTTTAGTAAAACTCTTAAACAGTGATTACA...,0.288561,0.261373,0.276908,0.161566,0.160309,0.290628,0.173784,LibA.Seq6744,0.094116,0.175465,0.295837,NaN,NaN,NaN,10,test
8490,0.164442,AGGACCGGATCAACTTTTTTATCTGGTTATCATTCTAGTCTAGTGC...,0.165896,0.184161,0.163828,0.106159,0.097415,0.181768,0.105090,LibA.Seq1298,0.116396,0.251474,0.155797,NaN,0.322232,0.272244,10,test


In [9]:
corr_coefficients

{'fold10': 1}

In [ ]:
from tensorflow.keras.optimizers import Adam

corr_coefficients = {}

# We iterate through each of the train folds to train, test and validate the model
for i in range(1,11):
    #Define inputs
    input_path_train = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_train.csv"
    input_path_valid = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_validation.csv"
    input_path_test = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/all_states_CV"+str(i)+"_LibA_wide_pivot_state3_test.csv"
    
    # Read test data to then predict
    df_test = pd.read_csv(input_path_test)
    df_test["fold"] = str(i)
    
    predictions_sum = np.zeros((df_test.shape[0], 7))
    
    # Get first item of the dataset to get the shape of the input data
    for element in data_reader(input_path_train):
        input_shape = element[0].shape
    
    # Define and compile model
    inputs = Input(shape=(input_shape[1],input_shape[2]), name="inputs")
    layer = Conv1D(250, kernel_size=7, strides=1, activation='relu', name="conv1")(inputs)  # 250 7 relu
    layer = Dropout(0.5)(layer)
    layer = BatchNormalization()(layer)
    layer = Conv1D(250, 8, strides=1, activation='softmax', name="conv2")(layer)  # 250 8 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=2, strides=None, name="maxpool1")(layer)
    layer = Dropout(0.3)(layer)
    layer = Conv1D(250, 3, strides=1, activation='softmax', name="conv3")(layer)  # 250 3 softmax
    layer = BatchNormalization()(layer)
    layer = Dropout(0.5)(layer)
    layer = Conv1D(100, 2, strides=1, activation='softmax', name="conv4")(layer)  # 100 3 softmax
    layer = BatchNormalization()(layer)
    layer = MaxPooling1D(pool_size=1, strides=None, name="maxpool2")(layer)
    layer = Dropout(0.5)(layer)
    layer = Flatten()(layer)
    layer = Dense(300, activation='sigmoid')(layer)  # 300
    layer = Dropout(0.5)(layer)
    layer = Dense(200, activation='sigmoid')(layer)  # 300
    predictions = Dense(7, activation='linear')(layer)
    norm_predictions = CustomNormalization()(predictions)  # Assuming "predictions" is your existing output

    model = Model(inputs=inputs, outputs=norm_predictions)
    model.summary()

    # compile model
    model.compile(optimizer= Adam(clipvalue=1.0),  # You can adjust the clipvalue as needed
                loss=MaskedMSE(mask_value=-1),
                metrics=["mse"]
                )

    # Run model
    history=model.fit(data_reader(input_path_train, batch_size=100),
                            epochs=20,
                            validation_data=data_reader(input_path_valid,batch_size=500),
                            callbacks=None,
                            verbose=1)    

    # We predict the test data
    predicted = model.predict(data_reader(input_path_test, batch_size=500))
    
    # We reed the data in the same order to compute the correlation score
    test_data = data_reader(input_path_test,batch_size=500)
    test_tensor = np.empty(shape=[0,7])
    for batch in test_data:
        test_tensor = np.append(test_tensor, batch[1], axis=0)
        
    # Append correlation coefficient and append to previous
    corr_coefficient = pearson_correlation_multi_head(test_tensor, predicted, mask_value=-1)
    avg_corr_coefficient = 1
        
    df_test["avg_prediction_State_1M"] = predicted[:,0]
    df_test["avg_prediction_State_2D"] = predicted[:,1]
    df_test["avg_prediction_State_3E"] = predicted[:,2]
    df_test["avg_prediction_State_4M"] = predicted[:,3]
    df_test["avg_prediction_State_5M"] = predicted[:,4]
    df_test["avg_prediction_State_6N"] = predicted[:,5]
    df_test["avg_prediction_State_7M"] = predicted[:,6]
    df_test["fold"] = str(i)
    df_test["partition"] = "test"
    corr_coefficients["fold"+str(i)] = avg_corr_coefficient
    
    # Append fold to previous folds
    df_test_10folds = pd.concat([df_test_10folds, df_test], ignore_index=True)
    break

# Save the results for all folds
df_test_10folds.to_csv("/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/ensemble_multihead_LibA_wide_pivot_state3_test_predicted_cv10fold.csv", index=False)

model_path = "/home/felix/cluster/fpacheco/Data/Robert_data/processed_data/10fold_cv/test_multihead_last_cv.h5"
model.save_weights(model_path, save_format='h5') 

df_test_10folds

In [ ]:
df_test_10folds
